In [3]:
import nltk 
import string 
from nltk.corpus import stopwords 

In [4]:
with open('positive_train_new.txt','r',encoding = 'utf8') as inFile, open('Filtered_positive_train_new.txt','w',encoding = 'utf8') as outFile: 
    for line in inFile.readlines(): 
        print(" ".join([word for word in line.lower().translate(str.maketrans('', '', string.punctuation)).split()  
            if len(word) >=4 and word not in stopwords.words('english')]), file=outFile) 

In [5]:
with open('negative_train_new.txt','r',encoding = 'utf8') as inFile, open('Filtered_negative_train_new.txt','w',encoding = 'utf8') as outFile: 
    for line in inFile.readlines(): 
        print(" ".join([word for word in line.lower().translate(str.maketrans('', '', string.punctuation)).split()  
            if len(word) >=4 and word not in stopwords.words('english')]), file=outFile) 

In [6]:
#Importing all the other necessary libraries first
import pandas, xgboost, numpy, textblob, string
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

In [7]:
#Reading the dataset
#There are two separate text files, containing the 2 classes of documents
data = open('Filtered_positive_train_new.txt',encoding="utf8").read()
data1 = open('Filtered_negative_train_new.txt',encoding="utf8").read() 

In [8]:
#There is no separate column denoting the label of each document.
#So adding label to each (Label_1 denotes violent; label_2 denotes violent)
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append(0)
#Joining elements from both the classes together into one single file 
    texts.append(" ".join(content[1:]))
for i, line in enumerate(data1.split("\n")):
    content = line.split()
    labels.append(1)
    texts.append(" ".join(content[1:]))

In [9]:
#Creating a dataframe using above texts and labels
import pandas
trainDF = pandas.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels

In [10]:
#Now, all instances of one class are clumped together
#Shuffling the rows of the dataframe to avoid discrepancies in future
from sklearn.utils import shuffle
trainDF = shuffle(trainDF)
trainDF

,text,label
3309,husband held charges cheating intimidation acc...,1
3204,vicechairman managing director infrastructure ...,1
3420,custody held charge isinspired group members d...,1
3979,departed souls traditional manners home attent...,1
1154,lynched manipurs imphal suspicion vehicle thef...,0
...,...,...
962,clash ramangaram followed stoning religious ka...,0
812,normalthe times india 1861current 1992 proques...,0
2411,drains covered 2019 0602 drainage lines perman...,1
3583,held killing husband teenage daughter also inv...,1


In [11]:
data = open('Times_January.txt',encoding="utf8").read()

In [12]:
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append("XX") 
    texts.append(" ".join(content[1:]))
    
#Creating a dataframe using above texts and labels
import pandas
testDF = pandas.DataFrame()
testDF['text'] = texts
testDF['label'] = labels
testDF

,text,label
0,Tripathy to debut in Sandalwood () Pitobash Tr...,XX
1,isn’t respect or fanfare: Yash on the fan who ...,XX
2,"on ‘noisy’ bikes beat up 2 policemen | Jan 12,...",XX
3,Ali Khan soaking in the sun will make you beam...,XX
4,rescues stranded tourists from north Sikkim in...,XX
...,...,...
13854,that propelled BJP to power in 2014 has dissip...,XX
13855,Delhi: Accused Mehtab arrested day after he ki...,XX
13856,"Nath may have been framed in rape case, observ...",XX
13857,"‘Can’t remove CM over pending cases’ | Jan 9, ...",XX


In [13]:
testDF = shuffle(testDF)

testDF1= testDF.iloc[0:1000]
result = trainDF.append(testDF1)
result

,text,label
3309,husband held charges cheating intimidation acc...,1
3204,vicechairman managing director infrastructure ...,1
3420,custody held charge isinspired group members d...,1
3979,departed souls traditional manners home attent...,1
1154,lynched manipurs imphal suspicion vehicle thef...,0
...,...,...
13776,protest in Kochi: Writing poems is ‘nun’ of he...,XX
7088,"Man posts private chats with ex on porn site, ...",XX
10501,I profited from paintings & I’ll quit: Mamata ...,XX
7050,Patel gets the temperature soaring with her la...,XX


In [14]:
trainDF = result
#Splitting the dataset into training(80%) and validation(20%) sets  
import numpy as np
from sklearn.model_selection import train_test_split
train_x, train_y=trainDF['text'].iloc[0:4968], trainDF['label'].iloc[0:4968]
valid_x, valid_y=trainDF['text'].iloc[4968:], trainDF['label'].iloc[4968:]

In [15]:
train_y

3309    1
3204    1
3420    1
3979    1
1154    0
       ..
962     0
812     0
2411    1
3583    1
4059    1
Name: label, Length: 4968, dtype: object

In [16]:
valid_y

8632     XX
5828     XX
8912     XX
11321    XX
13161    XX
         ..
13776    XX
7088     XX
10501    XX
7050     XX
1488     XX
Name: label, Length: 1000, dtype: object

In [17]:
#label encoding the class-labels or the target variable 
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [18]:
#The next step is the feature engineering step. 
#Raw text data will be transformed into feature vectors and new features will be created using the existing dataset.

#count vectorizer, tf-idf, word embeddings, topic models, part-of-speech tagging,

#Creating count vectorizer object and using it to transform data
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [19]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=3000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

C:\Users\ANKAN\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:501: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [20]:
#Using a pre-trained word embedding to create a mapping of token and their respective embeddings
import numpy
embeddings_index = {}
for i, line in enumerate(open('C:/Users/ANKAN/Downloads/wiki-news-300d-1M.vec/wiki-news-300d-1M.vec',encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

In [21]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers


#Creatng a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

#Converting text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=50)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=50)

Using TensorFlow backend.


In [22]:
# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [23]:
#Feature engineering - introducing new features that are expected to be useful
trainDF['char_count'] = trainDF['text'].apply(len)
trainDF['word_count'] = trainDF['text'].apply(lambda x: len(x.split()))
trainDF['word_density'] = trainDF['char_count'] / (trainDF['word_count']+1)
trainDF['punctuation_count'] = trainDF['text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
trainDF['title_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
trainDF['upper_case_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [24]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

In [25]:
# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

In [26]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ANKAN\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [27]:
#empty for now

In [28]:
trainDF.tail(10)

,text,label,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count
6987,roundabouts: Committee to check pricing of mat...,XX,1670,266,6.254682,24,24,1
2254,"SC defers hearing PIL against bill | Jan 15, 2...",XX,876,151,5.763158,32,31,11
7472,writer and professor of English Germaine Greer...,XX,2095,371,5.631720,57,44,5
155,10ha land needs to be acquired for Metro Phase...,XX,3709,639,5.795312,105,92,20
3441,Postcard campaign urges govt to back BMTC | Ja...,XX,2004,321,6.223602,45,37,11
13776,protest in Kochi: Writing poems is ‘nun’ of he...,XX,1981,350,5.643875,38,48,8
7088,"Man posts private chats with ex on porn site, ...",XX,710,119,5.916667,23,13,2
10501,I profited from paintings & I’ll quit: Mamata ...,XX,897,151,5.901316,21,27,6
7050,Patel gets the temperature soaring with her la...,XX,738,123,5.951613,18,11,0
1488,BSNL Bengaluru and the Union Bank of India hav...,XX,3086,513,6.003891,49,62,14


In [29]:
# train a LDA Model
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(xtrain_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

# view the topic models
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = numpy.array(vocab)[numpy.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

In [30]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [31]:
from sklearn.model_selection import cross_val_score

# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print ("Naive Bayes Count Vectors: ", accuracy)
scores = cross_val_score(naive_bayes.MultinomialNB(), xtrain_count, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')


# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("Naive Bayes WordLevel TF-IDF: ", accuracy)
scores = cross_val_score(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')


# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("Naive Bayes N-Gram Vectors: ", accuracy)
scores = cross_val_score(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')


# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("Naive Bayes CharLevel Vectors: ", accuracy)
scores = cross_val_score(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')


Naive Bayes Count Vectors:  0.789
10 fold Cross-validated score:  0.9808776043356916 

Naive Bayes WordLevel TF-IDF:  0.641
10 fold Cross-validated score:  0.9840997598494191 

Naive Bayes N-Gram Vectors:  1.0
10 fold Cross-validated score:  0.8780172973323814 

Naive Bayes CharLevel Vectors:  0.892
10 fold Cross-validated score:  0.9750413772960342 



In [32]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(max_iter=4000), xtrain_count, train_y, xvalid_count)
print ("Logistic Regression, Count Vectors: ", accuracy)
scores = cross_val_score(linear_model.LogisticRegression(max_iter=4000), xtrain_count, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("Logistic Regression, WordLevel TF-IDF: ", accuracy)
scores = cross_val_score(linear_model.LogisticRegression(max_iter=4000), xtrain_tfidf, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("Logistic Regression, N-Gram Vectors: ", accuracy)
scores = cross_val_score(linear_model.LogisticRegression(max_iter=4000), xtrain_tfidf_ngram, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("Logistic Regression, CharLevel Vectors: ", accuracy)
scores = cross_val_score(linear_model.LogisticRegression(max_iter=4000), xtrain_tfidf_ngram_chars, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')

Logistic Regression, Count Vectors:  0.001
10 fold Cross-validated score:  0.9883255176218603 

Logistic Regression, WordLevel TF-IDF:  0.008
10 fold Cross-validated score:  0.9913456545725969 

Logistic Regression, N-Gram Vectors:  0.306
10 fold Cross-validated score:  0.9553141429220483 

Logistic Regression, CharLevel Vectors:  0.108
10 fold Cross-validated score:  0.9800731810216137 



In [33]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("SVM, N-Gram Vectors: ", accuracy)
scores = cross_val_score(svm.SVC(), xtrain_tfidf_ngram, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')

SVM, N-Gram Vectors:  0.049
10 fold Cross-validated score:  0.9567242162653338 



In [34]:
# Random forests on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print ("Random Forests, Count Vectors: ", accuracy)
scores = cross_val_score(ensemble.RandomForestClassifier(), xtrain_count, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("Random Forests, WordLevel TF-IDF: ", accuracy)
scores = cross_val_score(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')

Random Forests, Count Vectors:  0.014
10 fold Cross-validated score:  0.9885283475043811 

Random Forests, WordLevel TF-IDF:  0.002
10 fold Cross-validated score:  0.988527536184851 



In [35]:
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print ("Xgb, Count Vectors: ", accuracy)
scores = cross_val_score(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')

# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print ("Xgb, WordLevel TF-IDF: ", accuracy)
scores = cross_val_score(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')

# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print ("Xgb, CharLevel Vectors: ", accuracy)
scores = cross_val_score(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')

Xgb, Count Vectors:  0.015
10 fold Cross-validated score:  0.9861114266242618 

Xgb, WordLevel TF-IDF:  0.006
10 fold Cross-validated score:  0.9853074089699488 

Xgb, CharLevel Vectors:  0.006
10 fold Cross-validated score:  0.9855098331927046 



In [65]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(xtrain_tfidf_ngram_chars, train_y)

LogisticRegression()

In [66]:
y_pred=lr.predict(xvalid_tfidf_ngram_chars)
y_pred

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [69]:
testDF1['predicted']=y_pred
df=testDF1.sort_values('predicted')
df.head(110)

<ipython-input-69-dc9592408b33>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testDF1['predicted']=y_pred


,text,label,predicted
8632,suspect held in Sahar firing case Mumbai: The ...,XX,0
10057,farmers killed over land dispute PATNA: Two fa...,XX,0
1543,fencing poses threat to black bucks in Haryana...,XX,0
6375,Poll rigging claim: Neta wants inquiry | Jan 2...,XX,0
12080,"boy crushed to death in Halol | Jan 6, 2019, 0...",XX,0
...,...,...,...
2948,"for gaving baby child, woman ends life | Jan 1...",XX,0
5188,refuses to marry drunk cop in Bhagalpur BHAGAL...,XX,0
11823,‘The Notebook’ to be made into Broadway musica...,XX,0
8314,() Television actress Kavita Kaushik and husba...,XX,1


In [70]:
df.to_csv('stage_2_predictions.csv')